In [12]:
import os
os.chdir("C:/Users/david/Desktop/Uni/potato-dry-matter-optics-ml")

from dotenv import load_dotenv

load_dotenv()  # lee el archivo .env

ROBOFLOW_API_KEY = os.environ["ROBOFLOW_API_KEY"]

# Comprobación rápida
print("ROBOFLOW_API_KEY configurada?:", "ROBOFLOW_API_KEY" in os.environ)

ROBOFLOW_API_KEY configurada?: True


In [13]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key=ROBOFLOW_API_KEY
)

In [14]:
from PIL import Image
from typing import List, Dict, Any, Tuple

def run_potato_detection(image_path: str):
    """
    Funciona con:
      - InferenceHTTPClient (JSON dict)
      - inference.get_model().infer (Pydantic objects)

    Devuelve:
      - raw_result
      - detections: [{class, confidence, bbox_xyxy}]
    """
    image = Image.open(image_path).convert("RGB")
    w, h = image.size

    # --- Inferencia por HTTP ---
    raw = CLIENT.infer(image_path, model_id="potato-detection-3et6q/11")
    raw_result = raw[0] if isinstance(raw, list) else raw

    detections: List[Dict[str, Any]] = []

    # Caso A) HTTP JSON dict
    if isinstance(raw_result, dict):
        predictions = raw_result.get("predictions", [])

        for det in predictions:
            cls = det.get("class")
            conf = float(det.get("confidence", 0.0))
            x = det.get("x")
            y = det.get("y")
            bw = det.get("width")
            bh = det.get("height")

            if None in (x, y, bw, bh):
                continue

            x1 = int(x - bw / 2)
            y1 = int(y - bh / 2)
            x2 = int(x + bw / 2)
            y2 = int(y + bh / 2)

            x1 = max(0, x1); y1 = max(0, y1)
            x2 = min(w, x2); y2 = min(h, y2)

            detections.append({
                "class": cls,
                "confidence": conf,
                "bbox_xyxy": (x1, y1, x2, y2),
            })

        return raw_result, detections

    # Caso B) Pydantic objects
    predictions = getattr(raw_result, "predictions", [])

    for det in predictions:
        cls = getattr(det, "class_name", None)
        conf = float(getattr(det, "confidence", 0.0))
        x = getattr(det, "x", None)
        y = getattr(det, "y", None)
        bw = getattr(det, "width", None)
        bh = getattr(det, "height", None)

        if None in (x, y, bw, bh):
            continue

        x1 = int(x - bw / 2)
        y1 = int(y - bh / 2)
        x2 = int(x + bw / 2)
        y2 = int(y + bh / 2)

        x1 = max(0, x1); y1 = max(0, y1)
        x2 = min(w, x2); y2 = min(h, y2)

        detections.append({
            "class": cls,
            "confidence": conf,
            "bbox_xyxy": (x1, y1, x2, y2),
        })

    return raw_result, detections

In [15]:
IMAGE_PATH = "data/input/raw/raw_images/test_0/p1_1.png"  # <-- cambia esto

raw_result, detections = run_potato_detection(IMAGE_PATH)

print("=== OUTPUT BRUTO DEL MODELO ===")
print(raw_result)  # Aquí ves TODO el JSON que devuelve el modelo

print("\n=== DETECCIONES PROCESADAS ===")
for i, det in enumerate(detections, start=1):
    print(f"Patata {i}:")
    print(f"  Clase (tipo): {det['class']}")
    print(f"  Confianza: {det['confidence']:.3f}")
    print(f"  Bounding box (x1, y1, x2, y2): {det['bbox_xyxy']}")


=== OUTPUT BRUTO DEL MODELO ===
{'inference_id': '8abb997f-c597-4356-9342-f5360eac9879', 'time': 0.36299968900857493, 'image': {'width': 3072, 'height': 2048}, 'predictions': [{'x': 1842.0, 'y': 997.5, 'width': 1182.0, 'height': 919.0, 'confidence': 0.5405731201171875, 'class': 'Damaged potato', 'class_id': 0, 'detection_id': '2fe591c1-6fb2-4ec6-b0d3-41c3c2678024'}]}

=== DETECCIONES PROCESADAS ===
Patata 1:
  Clase (tipo): Damaged potato
  Confianza: 0.541
  Bounding box (x1, y1, x2, y2): (1251, 538, 2433, 1457)


In [16]:
from PIL import Image, ImageDraw, ImageFont
import os

def save_detections_image(image_path: str, detections, output_path: str):
    # Cargar imagen
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)

    # Fuente para el texto
    try:
        font = ImageFont.truetype("arial.ttf", 18)
    except:
        font = ImageFont.load_default()

    for det in detections:
        x1, y1, x2, y2 = det["bbox_xyxy"]
        label = f"{det['class']} {det['confidence']:.2f}"

        # Caja
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)

        # Fondo para texto
        text_x, text_y = x1, max(0, y1 - 22)
        w_text = 8 * len(label)
        h_text = 20
        draw.rectangle(
            [text_x, text_y, text_x + w_text, text_y + h_text],
            fill="red"
        )

        # Texto
        draw.text((text_x + 2, text_y + 2), label, fill="white", font=font)

    # Crear carpeta si no existe
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    image.save(output_path)
    print(f"Imagen con detecciones guardada en: {output_path}")

In [17]:
OUTPUT_PATH = "data/input/processed_images/test_0/p1_1_def.png"

# Suponiendo que ya has hecho:
# raw_result, detections = run_potato_detection(IMAGE_PATH, confidence_threshold=0.4)

save_detections_image(IMAGE_PATH, detections, OUTPUT_PATH)


Imagen con detecciones guardada en: data/input/processed_images/test_0/p1_1_def.png
